In [1]:
pip install git+https://github.com/JustAnotherArchivist/snscrape.git

  Running command git clone -q https://github.com/JustAnotherArchivist/snscrape.git 'C:\Users\Utilisateur\AppData\Local\Temp\pip-req-build-dp4ambkm'



  Cloning https://github.com/JustAnotherArchivist/snscrape.git to c:\users\utilisateur\appdata\local\temp\pip-req-build-dp4ambkm
  Created wheel for snscrape: filename=snscrape-0.3.5.dev139+g35fb61a-py3-none-any.whl size=52226 sha256=e4227f93ff93970ccaa9dcede4f4343bb501fab728ed68aa984c1158b628b9d8
  Stored in directory: C:\Users\Utilisateur\AppData\Local\Temp\pip-ephem-wheel-cache-8eevl8aw\wheels\92\42\87\33fa9b18f7a75d02643a9ca3743339aec9be28c6796267c7d8
Successfully built snscrape


In [2]:
import time
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [19]:
start = time.time()

# On crée la liste qui contiendra les informations des tweets
tweets_list = []

# On utilise TwitterSearchScraper pour obtenir tous les tweets contenant un string choisi durant une période choisie
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('dune since:2021-09-20 until:2021-09-21 lang:en').get_items()):
    if i>1000: # On scrape 1000 tweets
        break
    tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.lang]) # On prend les informations intéressantes
    
# On crée un dataframe à partir de ce qu'on a scrapé
tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Language'])
end=time.time()
end-start

28.64079236984253

In [20]:
tweets_df

,Datetime,Tweet Id,Text,Username,Language
0,2021-09-20 23:59:54+00:00,1440103454702284801,"Wow, Final Jeopardy! was brutal. Category was ...",DEvanAltman,en
1,2021-09-20 23:59:18+00:00,1440103304533446658,I both love Dune and think it sucks because it...,flying_rodent,en
2,2021-09-20 23:59:08+00:00,1440103264670806018,"""God created Arrakis to train the Faithful ""\n...",_Joseph_Narvaez,en
3,2021-09-20 23:59:05+00:00,1440103250494201856,"@Kittenfish817 Coulda been shoulda been, alas ...",reverendmalibu,en
4,2021-09-20 23:59:00+00:00,1440103231200403459,"@MrWuggums No, we’ll have dunes full of booze!...",MittenheadSheri,en
...,...,...,...,...,...
996,2021-09-20 19:50:00+00:00,1440040566209716236,Dune was intense. I am in awe.,dogbikini,en
997,2021-09-20 19:49:48+00:00,1440040516494577664,@Aspire2Luminati @OkButStill I love that the d...,CorrosiveCummy,en
998,2021-09-20 19:49:35+00:00,1440040460282515466,Yeah #Dune https://t.co/ZJie5YAQh1,juanprogbass,en
999,2021-09-20 19:49:26+00:00,1440040425612464130,“ORNITHOPTER (commonly: ‘thopter): any aircraf...,WillsWaffleHut,en
